Unzipping folders(Please place folders into the working directory, outside sample_data)

In [2]:
from zipfile import ZipFile
for i in [1, 2, 4]:
  zip_file1=ZipFile(f"enron{i}_train.zip",'r')
  zip_file1.extractall()
  zip_file1.close()
  zip_file1=ZipFile(f"enron{i}_test.zip",'r')
  zip_file1.extractall()
  zip_file1.close()


Now manually make a folder for enron2 in the same way as enron1 and enron4 and place test and training for enron2 in there



---

Importing all the required libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


1. Bag of words model method

In [4]:
import os
import copy
from collections import Counter
import glob
import re
from math import log10 as log

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV


# This method is used to return the lists of strings from the txt files along with the sizes of those lists
def ImportDataset(Dataset, isTraining):

    hamFiles = []
    spamFiles = []
    totalFiles = ""

    path = os.path.join(os.getcwd(), Dataset)

    if isTraining == True:
        path = os.path.join(path, "train")
    else:
        path = os.path.join(path, "test")

    hamPaths=os.path.join(path,"ham")
    spamPaths=os.path.join(path,"spam")

    hamFileList = glob.glob(hamPaths + "/" + "*.txt")
    spamFileList = glob.glob(spamPaths+ "/" + "*.txt")

    for spamFile in spamFileList:
        spamFiles.append(open(spamFile, "r", encoding='Latin-1').read())
        totalFiles = totalFiles + " " + open(spamFile, "r",encoding='Latin-1').read()
    for hamFile in hamFileList:
        hamFiles.append(open(hamFile, "r",encoding='Latin-1').read())
        totalFiles = totalFiles + " " + open(hamFile, "r",encoding='Latin-1').read()





    sizeOfTotalFileList = len(hamFileList) + len(spamFileList)
    sizeOfHamList = len(hamFileList)
    sizeOfSpamList = len(spamFileList)
    return spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList

#This method converts the list of strings into a bag of word representation for spam and ham datasets respectively
def BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList):


    vocabulary = {}
    totalFrequencyList = {}
    spamFrequencyList = []
    spamVocab = {} #stores the spam files' word count
    hamVocab = {}#stores the ham files' word count
    hamFrequencyList = []


    totalFilteredData=re.findall("[a-zA-Z]+",totalFiles)
    stopwordsList = stopwords.words('english')

    for word in totalFilteredData:
        word = word.lower()
        if word in vocabulary:
            continue
        else:
            if word not in stopwordsList:
                vocabulary[word] = 0
        if word in totalFrequencyList:
            if word not in stopwordsList:
                totalFrequencyList[word] = totalFrequencyList[word] + 1
        else:
            if word not in stopwordsList:
                totalFrequencyList[word] = 1




  # spam files bag of words
    for email in spamFiles:
        spamFilteredData=re.findall("[a-zA-Z]+",email)
        tempDictionary = copy.deepcopy(vocabulary)
        for word in spamFilteredData:
            word = word.lower()
            if word in tempDictionary:
                tempDictionary[word] = tempDictionary[word] + 1
        spamVocab = Counter(spamVocab) + Counter(tempDictionary)
        spamFrequencyList.append(tempDictionary)

   # ham files bag of words
    for email in hamFiles:
        tempDictionary = copy.deepcopy(vocabulary)
        hamFilteredData=re.findall("[a-zA-Z]+",email)
        for word in hamFilteredData:
            word = word.lower()
            if word in tempDictionary:
                tempDictionary[word] = tempDictionary[word] + 1

        hamVocab = Counter(hamVocab) + Counter(tempDictionary)
        hamFrequencyList.append(tempDictionary)


    return spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary




def BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList):


    vocabulary = {}
    spamBernoulli = []
    spamVocab = {}
    hamBernoulli = []
    hamVocab = {}

    totalFilteredData=re.findall("[a-zA-Z]+",totalFiles)
    stopwordsList = stopwords.words('english')

    for word in totalFilteredData:
        word = word.lower()
        if word in vocabulary:
            continue
        else:
            if word not in stopwordsList:
                vocabulary[word] = 0


    for email in spamFiles:
        tempDictionary = copy.deepcopy(vocabulary)
        spamFilteredData=re.findall("[a-zA-Z]+",email)
        for word in spamFilteredData:
            word = word.lower()
            if word in tempDictionary:
                tempDictionary[word] = 1
                spamVocab[word] = 1
        temp_list = list(tempDictionary.values())
        spamBernoulli.append(tempDictionary)


    for email in hamFiles:
        tempDictionary = copy.deepcopy(vocabulary)
        hamFilteredData=re.findall("[a-zA-Z]+",email)
        for word in hamFilteredData:
            word = word.lower()
            if word in tempDictionary:
                tempDictionary[word] = 1
                hamVocab[word] = 1
        hamBernoulli.append(tempDictionary)
    return spamBernoulli, hamBernoulli, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary






#method to train MNB
def TrainMultinomialNB(spamFrequencyList, hamFrequencyList, totalFrequencyList,
                         spamVocab, hamVocab, sizeOfTotalFileList,
                         sizeOfSpamList, sizeOfHamList, vocabulary):



    condProbability = {}
    condProbability["spam"] = {}
    condProbability["ham"] = {}
    condProbabilityOfAbsentWord = {} #this is the conditional probabilities of words that may be absent from the spam/ham files in the training data
    condProbabilityOfAbsentWord["spam"] = {}
    condProbabilityOfAbsentWord["ham"] = {}
    priors = {}



    priors["spam"] = log(sizeOfSpamList / float(sizeOfTotalFileList))
    priors["ham"] =log(sizeOfHamList / float(sizeOfTotalFileList))
    totalSpamWords = sum(spamVocab.values())
    totalHamWords = sum(hamVocab.values())

    for word in list(spamVocab):
        condProbability["spam"][word] =log((spamVocab[word] + 1) / (float(totalSpamWords + len(totalFrequencyList))))


    for word in list(hamVocab):
        condProbability["ham"][word] =log((hamVocab[word] + 1) / (
            float(totalHamWords + len(totalFrequencyList))))

    condProbabilityOfAbsentWord["ham"] =log(1 / (float(totalHamWords + len(totalFrequencyList))))
    condProbabilityOfAbsentWord["spam"] =log(1 / (float(totalSpamWords + len(totalFrequencyList))))
    return condProbability, condProbabilityOfAbsentWord, priors

#method to test MNB, returns 1 if spam and -1 if ham
def TestMultinomialNB(priors, condProbability, condProbabilityOfAbsentWord,
                                 testingBOW):

    score = {}
    for _class in list(priors):
        score[_class] = priors[_class]
        for word in list(testingBOW):
            if testingBOW[word] != 0:
                try:
                    score[_class] += condProbability[_class][word]
                except KeyError:
                    score[_class] += condProbabilityOfAbsentWord[_class]

    if score["spam"] > score["ham"]:
        return 1
    else:
        return 0








def findAccuracy(predictedVal,trueVal):

    correctPrediction = 0
    for val in range(len(trueVal)):
        if trueVal[val] == predictedVal[val]:
            correctPrediction =correctPrediction+1
    return correctPrediction / float(len(trueVal))


def findPrecision(predictedVal,trueVal):

    truePositives = 0
    falsePositives = 0
    for val in range(len(trueVal)):
        if trueVal[val] == predictedVal[val] and predictedVal[val] == 1:
            truePositives += 1
        if trueVal[val] != predictedVal[val] and predictedVal[val] == 1:
            falsePositives += 1
    return truePositives / float(truePositives + falsePositives)


def findRecall(predictedVal,trueVal):

    truePositives = 0
    falseNegatives = 0
    for val in range(len(trueVal)):
        if trueVal[val] == predictedVal[val] and predictedVal[val] == 1:
            truePositives += 1
        if trueVal[val] != predictedVal[val] and predictedVal[val] == 0:
            falseNegatives += 1
    return truePositives / float(truePositives + falseNegatives)


def findF1_score(recall, precision):

    return (2 *precision*recall) / float(recall + precision)

















def TrainDiscreteNB(spamBernoulli, hamBernoulli,
                               spamVocab, hamVocab, sizeOfTotalFileList,
                               sizeOfSpamList, sizeOfHamList, vocabulary):




    condProbability = {}
    condProbability["spam"] = {}
    condProbability["ham"] = {}
    condProbabilityOfAbsentWord = {}
    condProbabilityOfAbsentWord["spam"] = {}
    condProbabilityOfAbsentWord["ham"] = {}
    priors = {}

    priors["spam"] =log(sizeOfSpamList/ float(sizeOfTotalFileList))
    priors["ham"] =log(sizeOfHamList / float(sizeOfTotalFileList))


    for word in spamVocab:
        condProbability["spam"][word] =log(
            1 + spamVocab[word] / (float(sizeOfSpamList+ 2)))

    for word in hamVocab:
        condProbability["ham"][word] =log(
            1 + hamVocab[word] / float(sizeOfHamList + 2))

    condProbabilityOfAbsentWord["ham"] =log(1 / (float(sizeOfHamList + 2)))
    condProbabilityOfAbsentWord["spam"] =log(1 / (float(sizeOfSpamList+ 2)))
    return condProbability, condProbabilityOfAbsentWord, priors


def TestDiscreteNB(priors, condProbability, condProbabilityOfAbsentWord,
                              testingBernoulli):

    score = {}

    for _class in list(priors):
        score[_class] = priors[_class]
        for word in list(testingBernoulli):
            if testingBernoulli[word] != 0:
                try:
                    score[_class] += condProbability[_class][word]

                except KeyError:
                    score[_class] += condProbabilityOfAbsentWord[_class]

    if score["spam"] > score["ham"]:
        return 1
    else:
        return 0









#to initialize W0 value and document class values

import random

def InitializeW0(spamMails,hamMails):
    for doc in spamMails:
        doc["W0"] = 1
        doc["DocumentClass"]=1
    for doc in hamMails:
        doc["W0"] = 1
        doc["DocumentClass"]=0
    return spamMails, hamMails

#functions to split training data
def DivideTrainingData(spamMails, hamMails):


    for doc in spamMails:
        doc["DocumentClass"] = 1
        doc["W0"] = 1

    for doc in hamMails:
        doc["DocumentClass"] = 0
        doc["W0"] = 1

    totalData = spamMails + hamMails
    random.shuffle(totalData)

    validation = totalData[0: int(len(totalData) * .3)]
    training = totalData[int(len(totalData) * .3): -1]

    return training, validation


#helper function to get output which is used to classify
def weightedSum(_input ,weight):

    res = weight['W0'] * 1
    for val in _input:
        if val == 'DocumentClass' or val == 'W0':
            continue
        else:
            if val in weight and val in _input:
                res = res + (_input[val] * weight[val])
    return res


def TrainMCAPLR(training, vocabulary, learningRate, _lambda, iterations):


    weight = copy.deepcopy(vocabulary)


    for val in weight:
        weight[val] = 0
    weight['W0'] = 0
    for val in range(iterations):
        for instance in training:
            try:
              posterior = 1/(float(1 + np.exp(-weightedSum(instance, weight))))
            except OverflowError:
              posterior = 1
            sum = 0
            for w in weight:
                if instance[w] != 0:
                    if w == 'W0':
                        sum = sum + learningRate * (
                                instance["DocumentClass"] - posterior)
                    else:
                        sum = sum + learningRate * (instance[w] * (instance["DocumentClass"] - posterior))
                    weight[w] = weight[w] + sum - learningRate * _lambda * weight[w]
    return weight


def TestMCAPLR(test_Input, weight):

    res = weightedSum(weight, test_Input)
    if res > 0:
        return 1
    else:
        return 0

#finding optimal lambda by iterating over lambda from 1 to 10 to see which one yields the highest accuracy using validation data
def ValidationMCAPLR(training, validation, vocabulary):

    learningRate = 0.1
    bestLambda = 2
    maxAccuracy = 0
    for _lambda in range(1, 10):
        correctPredictions = 0
        weight = TrainMCAPLR(training, vocabulary, learningRate, _lambda, 25)
        for value in validation:
            output = TestMCAPLR(value, weight)
            if output == value["DocumentClass"]:
                correctPredictions = correctPredictions + 1
        accuracy = correctPredictions / float(len(validation))
        if accuracy > maxAccuracy:
            maxAccuracy = accuracy
            bestLambda = _lambda
    return bestLambda











def InitializeDocClass(spamMails, hamMails):
    for doc in spamMails:
        doc["DocumentClass"] = 1
    for doc in hamMails:
        doc["DocumentClass"] = 0
    return spamMails, hamMails




def ParameterTuner(validationX, validationY):
    ParameterList = {'max_iter': (range(500, 2000, 1000)),'eta0': (0.2, 0.8),
                              'tol': (0.002, 0.004),'alpha': (0.01, 0.03), 'penalty': ('l1', 'l2')
                              }
    SGDclassifier = SGDClassifier()
    gridSearch = GridSearchCV(SGDclassifier, ParameterList, cv=5)
    gridSearch.fit(validationX, validationY)
    return gridSearch



def dataConverter(data, words):
    trainX = []
    trainY = []
    for value in data:
        trainXVal = []
        trainY.append(value["DocumentClass"])
        for word in words:
            try:
                trainXVal.append(value[word])
            except:
                trainXVal.append(0)
        trainX.append(trainXVal)
    return trainX, trainY

def TestSGDC(trainedClassifier, testX, testY):

    predictedVal = []
    for value in testX:
        predictedVal.append(trainedClassifier.predict(np.reshape(value, (1, -1))))
    return predictedVal, testY





def combineData(spamMails, hamMails):
    totalData = spamMails + hamMails
    return totalData


















---



**Results**



1. Bag of Words Multinomial NB



a) enron1

In [ ]:


spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList=ImportDataset("enron1",True)
#importing training data and creating bag of words
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)


#training MNB
condProbability, condProbabilityOfAbsentWord, priors = TrainMultinomialNB(spamFrequencyList, hamFrequencyList, totalFrequencyList,
                         spamVocab, hamVocab, sizeOfTotalFileList,
                         sizeOfSpamList, sizeOfHamList, vocabulary)



# #importing test data
spamFilesTest, hamFilesTest, totalFilesTest, sizeOfTotalFileListTest, sizeOfSpamListTest, sizeOfHamListTest = ImportDataset("enron1", False)


#creating bag of words with test data
spamFrequencyListTest, hamFrequencyListTest, totalFrequencyListTest, spamVocabTest, hamVocabTest, sizeOfTotalFileListTest, sizeOfSpamListTest, sizeOfHamListTest, vocabularyTest = BagOfWords(spamFilesTest, hamFilesTest, totalFilesTest, sizeOfTotalFileListTest, sizeOfSpamListTest, sizeOfHamListTest)
#testing MNB
predictedSpam = []
for value in spamFrequencyListTest:
    predictedSpam.append(TestMultinomialNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualSpam=[1]*len(predictedSpam)
predictedHam = []
for value in hamFrequencyListTest:
    predictedHam.append(TestMultinomialNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualHam=[0]*len(predictedHam)
totalActual=actualSpam+actualHam
totalPredicted=predictedSpam+ predictedHam
print("Accuracy: ",findAccuracy(totalPredicted,totalActual))
print("Precision: ",findPrecision(totalPredicted,totalActual))
print("Recall: ",findRecall(totalPredicted,totalActual))
print("F1 Score: ",findF1_score(findRecall(totalPredicted,totalActual),findPrecision(totalPredicted,totalActual)))

Accuracy:  0.9407894736842105
Precision:  0.8860759493670886
Recall:  0.9395973154362416
F1 Score:  0.9120521172638436


2) enron2, if running this gives you a divide by 0 error it means you have not made an enron2 folder and placed the train and test folders inside

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2", True)
#creating bag of words
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#training MNB
condProbability, condProbabilityOfAbsentWord, priors = TrainMultinomialNB(spamFrequencyList, hamFrequencyList, totalFrequencyList,
                         spamVocab, hamVocab, sizeOfTotalFileList,
                         sizeOfSpamList, sizeOfHamList, vocabulary)
#importing test data
spamFilesTest, hamFilesTest, totalFilesTest, sizeOfTotalFileListTest, sizeOfSpamListTest, sizeOfHamListTest = ImportDataset("enron1", False)
#creating bag of words with test data
spamFrequencyListTest, hamFrequencyListTest, totalFrequencyListTest, spamVocabTest, hamVocabTest, sizeOfTotalFileListTest, sizeOfSpamListTest, sizeOfHamListTest, vocabularyTest = BagOfWords(spamFilesTest, hamFilesTest, totalFilesTest, sizeOfTotalFileListTest, sizeOfSpamListTest, sizeOfHamListTest)
#testing MNB
predictedSpam = []
for value in spamFrequencyListTest:
    predictedSpam.append(TestMultinomialNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualSpam=[1]*len(predictedSpam)
predictedHam = []
for value in hamFrequencyListTest:
    predictedHam.append(TestMultinomialNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualHam=[0]*len(predictedHam)
totalActual=actualSpam+actualHam
totalPredicted=predictedSpam+predictedHam
print("Accuracy: ",findAccuracy(totalPredicted,totalActual))
print("Precision: ",findPrecision(totalPredicted,totalActual))
print("Recall: ",findRecall(totalPredicted,totalActual))
print("F1 Score: ",findF1_score(findRecall(totalPredicted,totalActual),findPrecision(totalPredicted,totalActual)))

Accuracy:  0.956140350877193
Precision:  0.910828025477707
Recall:  0.959731543624161
F1 Score:  0.934640522875817


3) enron4

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4", True)
#creating bag of words
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#training MNB
condProbability, condProbabilityOfAbsentWord, priors = TrainMultinomialNB(spamFrequencyList, hamFrequencyList, totalFrequencyList,
                         spamVocab, hamVocab, sizeOfTotalFileList,
                         sizeOfSpamList, sizeOfHamList, vocabulary)
#importing test data
spamFilesTest, hamFilesTest, totalFilesTest, sizeOfTotalFileListTest, sizeOfSpamListTest, sizeOfHamListTest = ImportDataset("enron1", False)
#creating bag of words with test data
spamFrequencyListTest, hamFrequencyListTest, totalFrequencyListTest, spamVocabTest, hamVocabTest, sizeOfTotalFileListTest, sizeOfSpamListTest, sizeOfHamListTest, vocabularyTest = BagOfWords(spamFilesTest, hamFilesTest, totalFilesTest, sizeOfTotalFileListTest, sizeOfSpamListTest, sizeOfHamListTest)
#testing MNB
predictedSpam = []
for value in spamFrequencyListTest:
    predictedSpam.append(TestMultinomialNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualSpam=[1]*len(predictedSpam)
predictedHam = []
for value in hamFrequencyListTest:
    predictedHam.append(TestMultinomialNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualHam=[0]*len(predictedHam)
totalActual=actualSpam+actualHam
totalPredicted=predictedSpam+predictedHam
print("Accuracy: ",findAccuracy(totalPredicted,totalActual))
print("Precision: ",findPrecision(totalPredicted,totalActual))
print("Recall: ",findRecall(totalPredicted,totalActual))
print("F1 Score: ",findF1_score(findRecall(totalPredicted,totalActual),findPrecision(totalPredicted,totalActual)))

Accuracy:  0.8355263157894737
Precision:  0.7371794871794872
Recall:  0.7718120805369127
F1 Score:  0.7540983606557377


2) Bernoulli Discrete NB

enron1

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1", True)
#creating bernoulli representation
spamBernoulli, hamBernoulli, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#training Discrete NB
condProbability, condProbabilityOfAbsentWord, priors = TrainDiscreteNB(spamBernoulli, hamBernoulli,
                               spamVocab, hamVocab, sizeOfTotalFileList,
                               sizeOfSpamList, sizeOfHamList, vocabulary)
#importing test data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1", False)
#creating bernoulli representation with test data
spamBernoulli, hamBernoulli, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#testing DNB
predictedSpam = []
for value in spamFrequencyListTest:
    predictedSpam.append(TestDiscreteNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualSpam=[1]*len(predictedSpam)
predictedHam = []
for value in hamFrequencyListTest:
    predictedHam.append(TestDiscreteNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualHam=[0]*len(predictedHam)
totalActual=actualSpam+actualHam
totalPredicted=predictedSpam+predictedHam
print("Accuracy: ",findAccuracy(totalPredicted,totalActual))
print("Precision: ",findPrecision(totalPredicted,totalActual))
print("Recall: ",findRecall(totalPredicted,totalActual))
print("F1 Score: ",findF1_score(findRecall(totalPredicted,totalActual),findPrecision(totalPredicted,totalActual)))

Accuracy:  0.9342105263157895
Precision:  0.8993288590604027
Recall:  0.8993288590604027
F1 Score:  0.8993288590604027


enron2

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2", True)
#creating bernoulli representation
spamBernoulli, hamBernoulli, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#training Discrete NB
condProbability, condProbabilityOfAbsentWord, priors = TrainDiscreteNB(spamBernoulli, hamBernoulli,
                               spamVocab, hamVocab, sizeOfTotalFileList,
                               sizeOfSpamList, sizeOfHamList, vocabulary)
#importing test data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2", False)
#creating bernoulli representation with test data
spamBernoulli, hamBernoulli, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#testing DNB
predictedSpam = []
for value in spamFrequencyListTest:
    predictedSpam.append(TestDiscreteNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualSpam=[1]*len(predictedSpam)
predictedHam = []
for value in hamFrequencyListTest:
    predictedHam.append(TestDiscreteNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualHam=[0]*len(predictedHam)
totalActual=actualSpam+actualHam
totalPredicted=predictedSpam+predictedHam
print("Accuracy: ",findAccuracy(totalPredicted,totalActual))
print("Precision: ",findPrecision(totalPredicted,totalActual))
print("Recall: ",findRecall(totalPredicted,totalActual))
print("F1 Score: ",findF1_score(findRecall(totalPredicted,totalActual),findPrecision(totalPredicted,totalActual)))

Accuracy:  0.9627192982456141
Precision:  0.9583333333333334
Recall:  0.9261744966442953
F1 Score:  0.9419795221843005


enron 4

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4", True)
#creating bernoulli representation
spamBernoulli, hamBernoulli, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#training Discrete NB
condProbability, condProbabilityOfAbsentWord, priors = TrainDiscreteNB(spamBernoulli, hamBernoulli,
                               spamVocab, hamVocab, sizeOfTotalFileList,
                               sizeOfSpamList, sizeOfHamList, vocabulary)
#importing test data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4", False)
#creating bernoulli representation with test data
spamBernoulli, hamBernoulli, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#testing DNB
predictedSpam = []
for value in spamFrequencyListTest:
    predictedSpam.append(TestDiscreteNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualSpam=[1]*len(predictedSpam)
predictedHam = []
for value in hamFrequencyListTest:
    predictedHam.append(TestDiscreteNB(priors, condProbability, condProbabilityOfAbsentWord, value))
actualHam=[0]*len(predictedHam)
totalActual=actualSpam+actualHam
totalPredicted=predictedSpam+predictedHam
print("Accuracy: ",findAccuracy(totalPredicted,totalActual))
print("Precision: ",findPrecision(totalPredicted,totalActual))
print("Recall: ",findRecall(totalPredicted,totalActual))
print("F1 Score: ",findF1_score(findRecall(totalPredicted,totalActual),findPrecision(totalPredicted,totalActual)))

Accuracy:  0.5986842105263158
Precision:  0.4472049689440994
Recall:  0.9664429530201343
F1 Score:  0.6114649681528662


3) MCAP Logistic Regression

3.1 Bag of words

enron1

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1", True)
#converting to BOW
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initialize w0
spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#splitting training data into training and validation
training, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)



In [ ]:
#training MCAP LR
bestLambda = ValidationMCAPLR(training, validation, vocabulary)
weight = TrainMCAPLR(training, vocabulary, 0.1, bestLambda, 100)

#importing testing data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1", False)
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)


<ipython-input-111-9f90305b466b>:384: RuntimeWarning: overflow encountered in exp
  return 1 / (float(1 + np.exp(-value)))


KeyError: 'W0'

In [ ]:
spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#testing weights
spamPredicted = []
for value in spamFrequencyList:
    spamPredicted.append(TestMCAPLR(value, weight))

hamPredicted = []
for value in hamFrequencyList:
    hamPredicted.append(TestMCAPLR(value, weight))

actualSpam = [1] * len(spamPredicted)
actualHam = [0] * len(hamPredicted)
totalActual = actualSpam + actualHam
totalPredicted = spamPredicted + hamPredicted

print("Accuracy: ", findAccuracy(totalPredicted, totalActual))
print("Precision: ", findPrecision(totalPredicted, totalActual))
print("Recall: ", findRecall(totalPredicted, totalActual))
print("F1 Score: ", findF1_score(findRecall(totalPredicted, totalActual), findPrecision(totalPredicted, totalActual)))


Accuracy:  0.9298245614035088
Precision:  0.8305084745762712
Recall:  0.9865771812080537
F1 Score:  0.901840490797546


enron2

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2", True)
#converting to BOW
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initialize w0
spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#splitting training data into training and validation
training, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)

#training MCAP LR
bestLambda = ValidationMCAPLR(training, validation, vocabulary)
weight = TrainMCAPLR(training, vocabulary, 0.1, bestLambda, 100)

#importing testing data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2", False)
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)

spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#testing weights
spamPredicted = []
for value in spamFrequencyList:
    spamPredicted.append(TestMCAPLR(value, weight))

hamPredicted = []
for value in hamFrequencyList:
    hamPredicted.append(TestMCAPLR(value, weight))

actualSpam = [1] * len(spamPredicted)
actualHam = [0] * len(hamPredicted)
totalActual = actualSpam + actualHam
totalPredicted = spamPredicted + hamPredicted

print("Accuracy: ", findAccuracy(totalPredicted, totalActual))
print("Precision: ", findPrecision(totalPredicted, totalActual))
print("Recall: ", findRecall(totalPredicted, totalActual))
print("F1 Score: ", findF1_score(findRecall(totalPredicted, totalActual), findPrecision(totalPredicted, totalActual)))



<ipython-input-111-9f90305b466b>:384: RuntimeWarning: overflow encountered in exp
  return 1 / (float(1 + np.exp(-value)))


Accuracy:  0.899581589958159
Precision:  0.735632183908046
Recall:  0.9846153846153847
F1 Score:  0.8421052631578947


enron4

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4", True)
#converting to BOW
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initialize w0
spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#splitting training data into training and validation
training, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)

#training MCAP LR
bestLambda = ValidationMCAPLR(training, validation, vocabulary)
weight = TrainMCAPLR(training, vocabulary, 0.1, bestLambda, 100)

#importing testing data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4", False)
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)

spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#testing weights
spamPredicted = []
for value in spamFrequencyList:
    spamPredicted.append(TestMCAPLR(value, weight))

hamPredicted = []
for value in hamFrequencyList:
    hamPredicted.append(TestMCAPLR(value, weight))

actualSpam = [1] * len(spamPredicted)
actualHam = [0] * len(hamPredicted)
totalActual = actualSpam + actualHam
totalPredicted = spamPredicted + hamPredicted

print("Accuracy: ", findAccuracy(totalPredicted, totalActual))
print("Precision: ", findPrecision(totalPredicted, totalActual))
print("Recall: ", findRecall(totalPredicted, totalActual))
print("F1 Score: ", findF1_score(findRecall(totalPredicted, totalActual), findPrecision(totalPredicted, totalActual)))

<ipython-input-115-b6a25ea57bd8>:387: RuntimeWarning: overflow encountered in exp
  posterior = 1/(float(1 + np.exp(-weightedSum(instance, weight))))


Accuracy:  0.6832412523020258
Precision:  0.9823788546255506
Recall:  0.5703324808184144
F1 Score:  0.7216828478964401


3.2 Bernoulli Model

enron1

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1", True)
#converting to Bernoulli

spamFrequencyList, hamFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initialize w0
spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#splitting training data into training and validation
training, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)

#training MCAP LR
bestLambda = ValidationMCAPLR(training, validation, vocabulary)
weight = TrainMCAPLR(training, vocabulary, 0.1, bestLambda, 100)

#importing testing data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1", False)
spamFrequencyList, hamFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)

spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#testing weights
spamPredicted = []
for value in spamFrequencyList:
    spamPredicted.append(TestMCAPLR(value, weight))

hamPredicted = []
for value in hamFrequencyList:
    hamPredicted.append(TestMCAPLR(value, weight))

actualSpam = [1] * len(spamPredicted)
actualHam = [0] * len(hamPredicted)
totalActual = actualSpam + actualHam
totalPredicted = spamPredicted + hamPredicted

print("Accuracy: ", findAccuracy(totalPredicted, totalActual))
print("Precision: ", findPrecision(totalPredicted, totalActual))
print("Recall: ", findRecall(totalPredicted, totalActual))
print("F1 Score: ", findF1_score(findRecall(totalPredicted, totalActual), findPrecision(totalPredicted, totalActual)))

<ipython-input-115-b6a25ea57bd8>:387: RuntimeWarning: overflow encountered in exp
  posterior = 1/(float(1 + np.exp(-weightedSum(instance, weight))))


Accuracy:  0.9210526315789473
Precision:  0.8228571428571428
Recall:  0.9664429530201343
F1 Score:  0.8888888888888888


enron2

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2", True)
#converting to Bernoulli

spamFrequencyList, hamFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initialize w0
spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#splitting training data into training and validation
training, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)

#training MCAP LR
bestLambda = ValidationMCAPLR(training, validation, vocabulary)
weight = TrainMCAPLR(training, vocabulary, 0.1, bestLambda, 100)

#importing testing data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2", False)
spamFrequencyList, hamFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)

spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#testing weights
spamPredicted = []
for value in spamFrequencyList:
    spamPredicted.append(TestMCAPLR(value, weight))

hamPredicted = []
for value in hamFrequencyList:
    hamPredicted.append(TestMCAPLR(value, weight))

actualSpam = [1] * len(spamPredicted)
actualHam = [0] * len(hamPredicted)
totalActual = actualSpam + actualHam
totalPredicted = spamPredicted + hamPredicted

print("Accuracy: ", findAccuracy(totalPredicted, totalActual))
print("Precision: ", findPrecision(totalPredicted, totalActual))
print("Recall: ", findRecall(totalPredicted, totalActual))
print("F1 Score: ", findF1_score(findRecall(totalPredicted, totalActual), findPrecision(totalPredicted, totalActual)))

<ipython-input-115-b6a25ea57bd8>:387: RuntimeWarning: overflow encountered in exp
  posterior = 1/(float(1 + np.exp(-weightedSum(instance, weight))))


Accuracy:  0.9121338912133892
Precision:  0.775
Recall:  0.9538461538461539
F1 Score:  0.8551724137931034


enron4

In [ ]:
#importing training data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4", True)
#converting to Bernoulli

spamFrequencyList, hamFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initialize w0
spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#splitting training data into training and validation
training, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)

#training MCAP LR
bestLambda = ValidationMCAPLR(training, validation, vocabulary)
weight = TrainMCAPLR(training, vocabulary, 0.1, bestLambda, 100)

#importing testing data
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4", False)
spamFrequencyList, hamFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)

spamFrequencyList,hamFrequencyList=InitializeW0(spamFrequencyList,hamFrequencyList)
#testing weights
spamPredicted = []
for value in spamFrequencyList:
    spamPredicted.append(TestMCAPLR(value, weight))

hamPredicted = []
for value in hamFrequencyList:
    hamPredicted.append(TestMCAPLR(value, weight))

actualSpam = [1] * len(spamPredicted)
actualHam = [0] * len(hamPredicted)
totalActual = actualSpam + actualHam
totalPredicted = spamPredicted + hamPredicted

print("Accuracy: ", findAccuracy(totalPredicted, totalActual))
print("Precision: ", findPrecision(totalPredicted, totalActual))
print("Recall: ", findRecall(totalPredicted, totalActual))
print("F1 Score: ", findF1_score(findRecall(totalPredicted, totalActual), findPrecision(totalPredicted, totalActual)))

<ipython-input-115-b6a25ea57bd8>:387: RuntimeWarning: overflow encountered in exp
  posterior = 1/(float(1 + np.exp(-weightedSum(instance, weight))))


Accuracy:  0.9337016574585635
Precision:  0.9156908665105387
Recall:  1.0
F1 Score:  0.9559902200488998


4) SGD Classifier from Scikit-learn

4.1 Bag of words model

enron1

In [5]:
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1",True)
#creating bag of words of training data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initializing document classes
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
trainData, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1",False)
#creating bag of words of testing data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
testData=combineData(spamFrequencyList,hamFrequencyList)
wordList=list(trainData[0])
trainX,trainY=dataConverter(trainData,wordList)
testX,testY=dataConverter(testData,wordList)
validationX,validationY=dataConverter(validation,wordList)

classifier=ParameterTuner(validationX,validationY)

trainedModel=classifier.fit(trainX, trainY)

predictedY, actualY=TestSGDC(trainedModel,testX,testY)
print("Accuracy: ",findAccuracy(predictedY,testY))
print("Precision: ",findPrecision(predictedY,testY))
print("Recall: ",findRecall(predictedY,testY))
print("F1 Score: ",findF1_score(findRecall(predictedY,testY),findPrecision(predictedY,testY)))


Accuracy:  0.9824561403508771
Precision:  0.9731543624161074
Recall:  0.9731543624161074
F1 Score:  0.9731543624161074


enron2

In [ ]:
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2",True)
#creating bag of words of training data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initializing document classes
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
trainData, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2",False)
#creating bag of words of testing data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
testData=combineData(spamFrequencyList,hamFrequencyList)
trainX,trainY=dataConverter(trainData,wordList)
testX,testY=dataConverter(testData,wordList)

validationX,validationY=dataConverter(validation,wordList)

classifier=ParameterTuner(validationX,validationY)

trainedModel=classifier.fit(trainX, trainY)

predictedY, actualY=TestSGDC(trainedModel,testX,testY)
print("Accuracy: ",findAccuracy(predictedY,testY))
print("Precision: ",findPrecision(predictedY,testY))
print("Recall: ",findRecall(predictedY,testY))
print("F1 Score: ",findF1_score(findRecall(predictedY,testY),findPrecision(predictedY,testY)))


Accuracy:  0.9853556485355649
Precision:  0.9624060150375939
Recall:  0.9846153846153847
F1 Score:  0.973384030418251


enron4

In [ ]:
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4",True)
#creating bag of words of training data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initializing document classes
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
trainData, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4",False)
#creating bag of words of testing data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
testData=combineData(spamFrequencyList,hamFrequencyList)
wordList=list(trainData[0])

trainX,trainY=dataConverter(trainData,wordList)
testX,testY=dataConverter(testData,wordList)
validationX,validationY=dataConverter(validation,wordList)

classifier=ParameterTuner(validationX,validationY)

trainedModel=classifier.fit(trainX, trainY)

predictedY, actualY=TestSGDC(trainedModel,testX,testY)
print("Accuracy: ",findAccuracy(predictedY,testY))
print("Precision: ",findPrecision(predictedY,testY))
print("Recall: ",findRecall(predictedY,testY))
print("F1 Score: ",findF1_score(findRecall(predictedY,testY),findPrecision(predictedY,testY)))


Accuracy:  0.992633517495396
Precision:  0.9898734177215189
Recall:  1.0
F1 Score:  0.9949109414758269


4.2 Bernoulli Model

enron1

In [ ]:
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1",True)
#creating bernoulli representation of training data
spamFrequencyList, hamFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initializing document classes
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
trainData, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron1",False)
#creating bag of words of testing data
spamFrequencyList, hamFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BernoulliModel(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
testData=combineData(spamFrequencyList,hamFrequencyList)
wordList=list(trainData[0])
trainX,trainY=dataConverter(trainData,wordList)
testX,testY=dataConverter(testData,wordList)
validationX,validationY=dataConverter(validation,wordList)

classifier=ParameterTuner(validationX,validationY)
#training the model based on the tuned parameters
trainedModel=classifier.fit(trainX, trainY)

predictedY, actualY=TestSGDC(trainedModel,testX,testY)
print("Accuracy: ",findAccuracy(predictedY,testY))
print("Precision: ",findPrecision(predictedY,testY))
print("Recall: ",findRecall(predictedY,testY))
print("F1 Score: ",findF1_score(findRecall(predictedY,testY),findPrecision(predictedY,testY)))


Accuracy:  1.0
Precision:  1.0
Recall:  1.0
F1 Score:  1.0


enron2

In [ ]:
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2",True)
#creating bag of words of training data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initializing document classes
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
trainData, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron2",False)
#creating bag of words of testing data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
testData=combineData(spamFrequencyList,hamFrequencyList)
wordList=list(trainData[0])
trainX,trainY=dataConverter(trainData,wordList)
testX,testY=dataConverter(testData,wordList)
validationX,validationY=dataConverter(validation,wordList)

classifier=ParameterTuner(validationX,validationY)

trainedModel=classifier.fit(trainX, trainY)

predictedY, actualY=TestSGDC(trainedModel,testX,testY)
print("Accuracy: ",findAccuracy(predictedY,testY))
print("Precision: ",findPrecision(predictedY,testY))
print("Recall: ",findRecall(predictedY,testY))
print("F1 Score: ",findF1_score(findRecall(predictedY,testY),findPrecision(predictedY,testY)))


Accuracy:  0.997907949790795
Precision:  1.0
Recall:  0.9923076923076923
F1 Score:  0.9961389961389961


enron4

In [ ]:
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4",True)
#creating bag of words of training data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
#initializing document classes
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
trainData, validation = DivideTrainingData(spamFrequencyList, hamFrequencyList)
spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList = ImportDataset("enron4",False)
#creating bag of words of testing data
spamFrequencyList, hamFrequencyList, totalFrequencyList, spamVocab, hamVocab, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList, vocabulary = BagOfWords(spamFiles, hamFiles, totalFiles, sizeOfTotalFileList, sizeOfSpamList, sizeOfHamList)
spamFrequencyList,hamFrequencyList=InitializeDocClass(spamFrequencyList,hamFrequencyList)
testData=combineData(spamFrequencyList,hamFrequencyList)
wordList=list(trainData[0])
trainX,trainY=dataConverter(trainData,wordList)
testX,testY=dataConverter(testData,wordList)
validationX,validationY=dataConverter(validation,wordList)

classifier=ParameterTuner(validationX,validationY)

trainedModel=classifier.fit(trainX, trainY)

predictedY, actualY=TestSGDC(trainedModel,testX,testY)
print("Accuracy: ",findAccuracy(predictedY,testY))
print("Precision: ",findPrecision(predictedY,testY))
print("Recall: ",findRecall(predictedY,testY))
print("F1 Score: ",findF1_score(findRecall(predictedY,testY),findPrecision(predictedY,testY)))


Accuracy:  0.9705340699815838
Precision:  0.9629629629629629
Recall:  0.9974424552429667
F1 Score:  0.9798994974874371
